## ROE指标

ROE是评价公司质量效率最高的因子。相比于企业的利润增长幅度，ROE指标更加稳定，且更接近公司创造价值的本质。


正常来讲，公司的归母利润增速应该为
> ROE * (1 - 分红率) 

当出现增发等情况时，应该更改为
> ROE * (净资本扩大系数)

当然这只是一个粗略的计算，会计准则上对净资产还有若干种处理方法

如果公司利润出现了暴增，那么有几种可能：
1. ROE剧增
2. 净资产扩大
由于是相乘的关系，所以当2项同时增加时，增速会表现为暴增


之后对ROE进行更深入的分析，ROE可以拆解为3个指标
> ROE = 周转率 * 净利率 * 杠杆率

这3个指标都与企业的经营息息相关，按照我的理解，周转率主要与需求和运行效率有关，净利率同样，杠杆率影响企业生产力

在企业的正常经营过程中，这3个指标也不太会突变，如果突变，一定是有大事件影响，所以我们对这3个指标也进行逐一分析


最后选择历史上最高的ROE之一来看看

### 做这个研究的目的

因为有一些企业依靠增发来获得更高的利润增速，这样的增速既不能等同于原股东权益的增速，又不可持续。

这里就统计全A公司，20年来，各年度ROE的分布情况，以及所有公司20年间的平均ROE分布情况。所得到的统计结果能用于判断，我们是否对某些公司的未来发展寄予了过高的期望。


In [ ]:
from jqfactor import get_factor_values
from jqdata import  get_trade_days
import numpy as np

# 对factor因子过去15年按年度的指标进行统计分析
def stas_factor(factor, report = 'year', num = 15):
    
    # 当前security数量为4539，超过3000上限，所以分为2部分
    security = get_all_securities().index.tolist()
    security_1 = security[0:3000]
    security_2 = security[3000:]
    
    factor_list = []
    max_factor = []
    
    for ii in np.arange(2006, 2021, 1):
        # 选择年报公布截止日后15天
        date = str(ii) + '-5-15'
        date = get_trade_days(end_date = date, count = 1)[0]
        
        fac_values = pd.DataFrame()
        for security in [security_1, security_2]:
            # roe_ttm, net_profit_ratio， 1/equity_to_asset_ratio， total_asset_turnover_rate
            fac_values_t = get_factor_values(securities=security, factors=factor,
                                            start_date=date, end_date=date)
            # col_name: securities, row_name: date
            fac_values_t = fac_values_t[factor]
            # assert fac_values.shape[0] == 1
            fac_values = pd.concat([fac_values, fac_values_t], axis = 1)

            # 获取每一年的最大指标，注意要对因子取倒数得到杠杆率
            # for index, row in fac_values.iterrows():
        if not factor == 'equity_to_asset_ratio':
            max_factor.append( [date, fac_values.idxmax(axis=1), fac_values.max(axis=1)] )
            tmp = 1/np.array(fac_values.iloc[0:,].dropna().tolist())
            factor_list += tmp.tolist()
        else:
            max_factor.append( [date, fac_values.idxmin(axis=1), 1 / fac_values.min(axis=1)] )
            factor_list += fac_values.iloc[0,:].dropna().tolist()

        # print('number of valid factor: ', date, fac_values.iloc[0,:].dropna().tolist())

    return [factor_list, max_factor]

In [ ]:
%%time

import pandas as pd
import numpy as np

# ROE，净利润率，权益资产比，总资产周转率
fac_list = ['roe_ttm', 'net_profit_ratio', 'equity_to_asset_ratio', 'total_asset_turnover_rate']

values_list = {}
maxv_list = {}

for factor in fac_list:
    [fac_values, max_fac] = stas_factor(factor)
    values_list[factor] = fac_values
    maxv_list[factor] = max_fac


### 结果说明
两个变量：

- values_list: 每个指标在过去15年，所有股票的年度值的列表，用于后续统计分析
- maxv_list： 每个指标，每年最大值对应的股票代码，用于特例分析

In [ ]:
def analysis_fac(values_list):
    
    # 注意不要提前定义名字，否则在concat操作时会对比改名字与tmp变量的列名，导致无法比较错误
    result = pd.DataFrame()
    # 构造最大、最小、平均、中位、上四分位值
    for fac in values_list:
        df_tmp = pd.DataFrame([[fac, np.max(values_list[fac]), np.min(values_list[fac]), np.median(values_list[fac]),
                                np.percentile(values_list[fac], 75), np.percentile(values_list[fac], 95)]])
        result = pd.concat([result, df_tmp], axis = 0, ignore_index=True)
    result.columns = ['FACTOR','MAX','MIN','MEDIAN','%75 Perc','95% Perc']
    
    if 'equity_to_asset_ratio' in result['FACTOR']:
        result.iloc[result['FACTOR'] == 'equity_to_asset_ratio',0] == 'leverage rate'
    
    return result
    
# 分析各指标的最大值在各年度对应的股票名称，杠杆率的分析应该没啥意义，杠杆率越高一般越不好
def analysis_max_fac(maxv):
    
    all_info = get_all_securities()
    # 得出每年最大值对应股票和相关数据
    for key in maxv:
        maxv[key]
    
    maxv = maxv.join(on='code')
    return
    

In [ ]:
ran = np.arange(-0.5,1,0.02)
cnt = plt.hist(values_list['roe_ttm'], bins=ran,density=True)

In [ ]:
aaa = analysis_fac(values_list)
aaa

### 结果分析
1. ROE

2. 净利润率

3. 杠杆率

4. 总资产周转率

4个指标的分布总体上都服从正态分布，但分布范围都及其广，ROE最高能有1000多，也就是100000%，这是我统计之前没想到的，但中位数是6%，另外95%分位数是，也就是说假设市场上只有5%值得投资的公司，那它的ROE门槛应该在XX，其实真实的门槛还要降，以剔除各年度的妖怪值。

In [ ]:
analysis_max_fac


### 极值分析
粗略看一下15年间ROE的极值对应的公司，及其杜邦分析的几个指标
总的看，xxx
其中xxx股票xxx，其10年平均roe是多少，观察是否有极值roe和均值roe的明显背离
所以，极值都


In [ ]:
def get_avg_roe():
    

### 总结
优秀的公司，它的ROE应该如何，应该像茅台一样，持续多年20%以上，已经非常不易，幻想一个公司在不增发的情况下能维持xxx速度的增长非常不现实

### BUG
1. get_trade_days只对2005年及以后有效
    > date = '2005-5-15' 
    > date = get_trade_days(end_date = date, count = 1)
    > date
    > None
2. factor支持的查询最大数量是3000，也就是股票数*日期数*因子数，单次不能超过3000